In [4]:
import json

import bz2
from tqdm import tqdm
from scipy import sparse

In [5]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:30, 2215.92it/s]


#  Домашнее задание по NLP # 2 [100 баллов] 
## Составление словарей для классификации по тональности
При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.  

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).


Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

In [4]:
responses[99]

{'author': 'ronnichka',
 'bank_license': 'лицензия № 880',
 'bank_name': 'Югра',
 'city': 'г. Саратов',
 'datetime': '2015-06-03 20:56:57',
 'num_comments': 0,
 'rating_grade': 3,
 'rating_not_checked': False,
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'title': 'Карта ко вкладу'}

## Часть 1. Обучение модели word2vec [50 баллов]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.  
 


Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

### 1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 

In [1]:
from nltk.tokenize import sent_tokenize

In [2]:
# text = 'Hello, world! How are you? i am fine'
# sents = sent_tokenize(text)
# sents

In [3]:
from pymystem3 import Mystem
import re


m = Mystem()


regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""



def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "


In [98]:
#df = pd.DataFrame.from_dict(responses)

In [48]:
df = pd.read_csv("banki_responses.csv")

In [49]:
#df.to_csv("banki_responses.csv")

In [50]:
df= df.iloc[:1000]

разбиваем на предложения

In [51]:
sentences = []
for i in range(len(df)):
    sentences.extend(sent_tokenize(df.text.iloc[i]))

лемматезируем

In [52]:
sentences = [lemmatize(sentence) for sentence in sentences]

готовим слова к Word2Vec

In [53]:
texts = [sentence.split() for sentence in sentences]

### 2. Обучите по коллекции предложений word2vec

In [55]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)

CPU times: user 1.41 s, sys: 21 ms, total: 1.43 s
Wall time: 573 ms


### 3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:

###     * тест на определение ближайших слов

In [56]:
model.most_similar("счет")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('денежный', 0.9940987229347229),
 ('средство', 0.9937266707420349),
 ('договор', 0.9937083721160889),
 ('ставка', 0.99346524477005),
 ('пополнение', 0.9930682182312012),
 ('кредитный', 0.9923174381256104),
 ('сумма', 0.9918913841247559),
 ('на', 0.9908804893493652),
 ('платеж', 0.9902487993240356),
 ('кредит', 0.9901759028434753)]

###    * тест на аналогии (мужчина – король : женщина – королева)

In [57]:
model.most_similar(positive=["счет","мой"], negative=["менеджер"])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('средство', 0.9895479679107666),
 ('сумма', 0.9854636192321777),
 ('кредитный', 0.9840105772018433),
 ('денежный', 0.9817023873329163),
 ('по', 0.9809874296188354),
 ('договор', 0.9778134226799011),
 ('ставка', 0.9777290225028992),
 ('вклад', 0.9772037267684937),
 ('на', 0.9767388105392456),
 ('пополнение', 0.9699004292488098)]

###    * тест на определение лишнего слова.

In [58]:
model.doesnt_match("карта счет банк быть".split())

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'быть'